## Part A: Data Cleaning & KPI Logic

In [4]:
#Import libraries
import pandas as pd

#Load Excel files
purchase_df = pd.read_excel("C:\\Users\\VISHAL\\Fibre Bond Industries\\dalbirsingh_test_pack\\purchase_register.xlsx")
production_df = pd.read_excel("C:\\Users\\VISHAL\\Fibre Bond Industries\\dalbirsingh_test_pack\\production_batchwise.xlsx")
kpi_df = pd.read_excel("C:\\Users\\VISHAL\\Fibre Bond Industries\\dalbirsingh_test_pack\\kpi_reference.xlsx")


In [5]:
#pip install openpyxl

In [6]:
purchase_df

,Purchase_ID,Batch_ID,Material,Qty,Unit_Cost,Date,Total_Cost
0,P1,B100,PU Resin,124,68.11,2025-04-01,8445.64
1,P2,B101,PVC,127,70.05,2025-04-02,8896.35
2,P3,B102,Colorant,196,99.39,2025-04-03,19480.44
3,P4,B103,Fabric,89,64.98,2025-04-04,5783.22
4,P5,B104,Adhesive,71,93.46,2025-04-05,6635.66
5,P6,B105,Solvent,123,82.94,2025-04-06,10201.62


In [7]:
production_df

,Batch_ID,Output_Units,Output_Date
0,B100,1220,2025-04-05
1,B101,978,2025-04-06
2,B102,831,2025-04-07
3,B103,1084,2025-04-08
4,B104,1386,2025-04-09
5,B105,1336,2025-04-10


In [8]:
kpi_df

,Batch_ID,Target_Cost_Per_Unit
0,B100,75.19
1,B101,79.15
2,B102,72.90
3,B103,61.32
4,B104,61.58
5,B105,82.29


In [9]:
#Step 1: Aggregate total raw material cost per batch
total_cost_per_batch = purchase_df.groupby("Batch_ID")["Total_Cost"].sum().reset_index()
total_cost_per_batch.rename(columns={"Total_Cost": "Total_Raw_Material_Cost"}, inplace=True)

In [10]:
total_cost_per_batch

,Batch_ID,Total_Raw_Material_Cost
0,B100,8445.64
1,B101,8896.35
2,B102,19480.44
3,B103,5783.22
4,B104,6635.66
5,B105,10201.62


In [11]:
total_cost_per_batch

,Batch_ID,Total_Raw_Material_Cost
0,B100,8445.64
1,B101,8896.35
2,B102,19480.44
3,B103,5783.22
4,B104,6635.66
5,B105,10201.62


In [12]:
#Step 2: Merge with production data
merged_df = pd.merge(total_cost_per_batch, production_df, on="Batch_ID", how="inner")
merged_df

,Batch_ID,Total_Raw_Material_Cost,Output_Units,Output_Date
0,B100,8445.64,1220,2025-04-05
1,B101,8896.35,978,2025-04-06
2,B102,19480.44,831,2025-04-07
3,B103,5783.22,1084,2025-04-08
4,B104,6635.66,1386,2025-04-09
5,B105,10201.62,1336,2025-04-10


In [13]:
#Step 3: Merge with KPI reference
merged_df = pd.merge(merged_df, kpi_df, on="Batch_ID", how="inner")
merged_df

,Batch_ID,Total_Raw_Material_Cost,Output_Units,Output_Date,Target_Cost_Per_Unit
0,B100,8445.64,1220,2025-04-05,75.19
1,B101,8896.35,978,2025-04-06,79.15
2,B102,19480.44,831,2025-04-07,72.90
3,B103,5783.22,1084,2025-04-08,61.32
4,B104,6635.66,1386,2025-04-09,61.58
5,B105,10201.62,1336,2025-04-10,82.29


In [14]:
#Step 4: Calculate cost per unit and variance
merged_df["Cost_Per_Unit"] = merged_df["Total_Raw_Material_Cost"] / merged_df["Output_Units"]
merged_df["Variance_%"] = ((merged_df["Cost_Per_Unit"] - merged_df["Target_Cost_Per_Unit"]) /
                           merged_df["Target_Cost_Per_Unit"]) * 100
merged_df

,Batch_ID,Total_Raw_Material_Cost,Output_Units,Output_Date,Target_Cost_Per_Unit,Cost_Per_Unit,Variance_%
0,B100,8445.64,1220,2025-04-05,75.19,6.922656,-90.793116
1,B101,8896.35,978,2025-04-06,79.15,9.096472,-88.507300
2,B102,19480.44,831,2025-04-07,72.90,23.442166,-67.843394
3,B103,5783.22,1084,2025-04-08,61.32,5.335074,-91.299619
4,B104,6635.66,1386,2025-04-09,61.58,4.787633,-92.225343
5,B105,10201.62,1336,2025-04-10,82.29,7.635943,-90.720691


In [15]:
#Step 5: Flag rows with high variance
merged_df["Variance_Flag"] = merged_df["Variance_%"].apply(
    lambda x: "High Variance" if abs(x) > 10 else "Within Range"
)
merged_df

,Batch_ID,Total_Raw_Material_Cost,Output_Units,Output_Date,Target_Cost_Per_Unit,Cost_Per_Unit,Variance_%,Variance_Flag
0,B100,8445.64,1220,2025-04-05,75.19,6.922656,-90.793116,High Variance
1,B101,8896.35,978,2025-04-06,79.15,9.096472,-88.507300,High Variance
2,B102,19480.44,831,2025-04-07,72.90,23.442166,-67.843394,High Variance
3,B103,5783.22,1084,2025-04-08,61.32,5.335074,-91.299619,High Variance
4,B104,6635.66,1386,2025-04-09,61.58,4.787633,-92.225343,High Variance
5,B105,10201.62,1336,2025-04-10,82.29,7.635943,-90.720691,High Variance


In [16]:
#Optional: Save cleaned data to CSV
merged_df.to_csv("cleaned_kpi_output.csv", index=False)


In [17]:
#Show final output
merged_df.head()


,Batch_ID,Total_Raw_Material_Cost,Output_Units,Output_Date,Target_Cost_Per_Unit,Cost_Per_Unit,Variance_%,Variance_Flag
0,B100,8445.64,1220,2025-04-05,75.19,6.922656,-90.793116,High Variance
1,B101,8896.35,978,2025-04-06,79.15,9.096472,-88.507300,High Variance
2,B102,19480.44,831,2025-04-07,72.90,23.442166,-67.843394,High Variance
3,B103,5783.22,1084,2025-04-08,61.32,5.335074,-91.299619,High Variance
4,B104,6635.66,1386,2025-04-09,61.58,4.787633,-92.225343,High Variance


## Part B: Query Parser

In [18]:
#Import libraries
import re
from datetime import datetime

In [19]:
#Step 1: Convert Output_Date to datetime and extract month name
merged_df["Output_Date"] = pd.to_datetime(merged_df["Output_Date"])
merged_df["Month"] = merged_df["Output_Date"].dt.strftime("%B")
merged_df

,Batch_ID,Total_Raw_Material_Cost,Output_Units,Output_Date,Target_Cost_Per_Unit,Cost_Per_Unit,Variance_%,Variance_Flag,Month
0,B100,8445.64,1220,2025-04-05,75.19,6.922656,-90.793116,High Variance,April
1,B101,8896.35,978,2025-04-06,79.15,9.096472,-88.507300,High Variance,April
2,B102,19480.44,831,2025-04-07,72.90,23.442166,-67.843394,High Variance,April
3,B103,5783.22,1084,2025-04-08,61.32,5.335074,-91.299619,High Variance,April
4,B104,6635.66,1386,2025-04-09,61.58,4.787633,-92.225343,High Variance,April
5,B105,10201.62,1336,2025-04-10,82.29,7.635943,-90.720691,High Variance,April


In [20]:
#Step 2: Define GPT-style query filter function
#Features Supported:
#cost/unit > or < or = number
#Month name
#"high variance" or "within range" keywords

def filter_batches_by_query(query):
    df = merged_df.copy()

    # Lowercase the query
    query = query.lower()

    # Filter by cost per unit
    #Only for Decimal Value ( 'cost_match = re.search(r"cost/?unit\s*([<>=]+)\s*(\d+)", query)' )
    # cost_match = re.search(r"cost/?unit\s*([<>=]+)\s*(\d+(?:\.\d+)?)", query)

    # if cost_match:
    #     op = cost_match.group(1)
    #     value = float(cost_match.group(2))
    #     df = df.query(f"Cost_Per_Unit {op} @value")
    # Filter by cost per unit
    cost_match = re.search(r"cost/?unit\s*([<>=]+)\s*(\d+(?:\.\d+)?)", query)
    if cost_match:
        op = cost_match.group(1)
        value = float(cost_match.group(2))
        if op == "=" or op == "==":
            df = df[df["Cost_Per_Unit"] == value]
        else:
            df = df.query(f"Cost_Per_Unit {op} @value")


    # Filter by month
    month_match = re.search(r"(january|february|march|april|may|june|july|august|september|october|november|december)", query)
    if month_match:
        month = month_match.group(1).capitalize()
        df = df[df["Month"] == month]

    # Filter by variance flag
    if "high variance" in query:
        df = df[df["Variance_Flag"] == "High Variance"]
    elif "within range" in query:
        df = df[df["Variance_Flag"] == "Within Range"]

    return df

#### Batches with Cost_Per_Unit	higher than 8 in April

In [21]:
#Example Queries
example_1 = filter_batches_by_query("Batches with cost/unit > 8 in April")
example_1

,Batch_ID,Total_Raw_Material_Cost,Output_Units,Output_Date,Target_Cost_Per_Unit,Cost_Per_Unit,Variance_%,Variance_Flag,Month
1,B101,8896.35,978,2025-04-06,79.15,9.096472,-88.507300,High Variance,April
2,B102,19480.44,831,2025-04-07,72.90,23.442166,-67.843394,High Variance,April


#### Batches with Cost_Per_Unit	lower than 8 in April

In [22]:
#Example Queries
example_1 = filter_batches_by_query("Batches with cost/unit < 8 in April")
example_1

,Batch_ID,Total_Raw_Material_Cost,Output_Units,Output_Date,Target_Cost_Per_Unit,Cost_Per_Unit,Variance_%,Variance_Flag,Month
0,B100,8445.64,1220,2025-04-05,75.19,6.922656,-90.793116,High Variance,April
3,B103,5783.22,1084,2025-04-08,61.32,5.335074,-91.299619,High Variance,April
4,B104,6635.66,1386,2025-04-09,61.58,4.787633,-92.225343,High Variance,April
5,B105,10201.62,1336,2025-04-10,82.29,7.635943,-90.720691,High Variance,April


In [23]:
# Example Queries
example_2 = filter_batches_by_query("Show batches with high variance in April")
example_2

,Batch_ID,Total_Raw_Material_Cost,Output_Units,Output_Date,Target_Cost_Per_Unit,Cost_Per_Unit,Variance_%,Variance_Flag,Month
0,B100,8445.64,1220,2025-04-05,75.19,6.922656,-90.793116,High Variance,April
1,B101,8896.35,978,2025-04-06,79.15,9.096472,-88.507300,High Variance,April
2,B102,19480.44,831,2025-04-07,72.90,23.442166,-67.843394,High Variance,April
3,B103,5783.22,1084,2025-04-08,61.32,5.335074,-91.299619,High Variance,April
4,B104,6635.66,1386,2025-04-09,61.58,4.787633,-92.225343,High Variance,April
5,B105,10201.62,1336,2025-04-10,82.29,7.635943,-90.720691,High Variance,April


## Manuall input comment to get accurate result as output, Kindly run below code each time to add new query


In [ ]:
# 📥 Get user input query
user_query = input("Enter your query (e.g., 'Batches with cost/unit > 8 in April'): ")

# 🔍 Filter based on the query
user_result = filter_batches_by_query(user_query)

# 🖨️ Show result
if not user_result.empty:
    display(user_result)
else:
    print("No matching records found.")

##Example in SQL 
#Enter your query (e.g.,  'Show batches with high variance in April')

##Example in java in input box as
#Enter your query (e.g., 'Batches with cost/unit > 8 in April'):


### Global Query Parser Function with Input Method

In [ ]:
# Global query Function
import re
from datetime import datetime

# Ensure Output_Date and Month are properly set
merged_df["Output_Date"] = pd.to_datetime(merged_df["Output_Date"])
merged_df["Month"] = merged_df["Output_Date"].dt.strftime("%B")

# 🔍 GPT-style query handler
def filter_batches_by_query(query):
    df = merged_df.copy()
    query = query.lower()

    # Cost per unit condition
    cost_match = re.search(r"cost/?unit\s*([<>=]=?|=)\s*(\d+(?:\.\d+)?)", query)
    if cost_match:
        op = cost_match.group(1)
        value = float(cost_match.group(2))
        if op in ["=", "=="]:
            df = df[df["Cost_Per_Unit"] == value]
        else:
            df = df.query(f"Cost_Per_Unit {op} @value")

    # Target cost per unit condition
    target_match = re.search(r"target.*between\s*(\d+(?:\.\d+)?)\s*to\s*(\d+(?:\.\d+)?)", query)
    if target_match:
        low = float(target_match.group(1))
        high = float(target_match.group(2))
        df = df[(df["Target_Cost_Per_Unit"] >= low) & (df["Target_Cost_Per_Unit"] <= high)]

    # Variance % range condition
    variance_range = re.search(r"variance.*between\s*(-?\d+(?:\.\d+)?)\s*to\s*(-?\d+(?:\.\d+)?)", query)
    if variance_range:
        low = float(variance_range.group(1))
        high = float(variance_range.group(2))
        df = df[(df["Variance_%"] >= low) & (df["Variance_%"] <= high)]

    # Month condition
    month_match = re.search(
        r"\b(january|february|march|april|may|june|july|august|september|october|november|december)\b",
        query,
    )
    if month_match:
        month = month_match.group(1).capitalize()
        df = df[df["Month"] == month]

    # Variance flag keywords
    if "high variance" in query:
        df = df[df["Variance_Flag"] == "High Variance"]
    elif "within range" in query:
        df = df[df["Variance_Flag"] == "Within Range"]

    return df

### Result for input Query " Show batches with target cost between 70 to 90 in April "
 

In [ ]:
# Interactive input in Jupyter
user_query = input("Enter your query (e.g., 'Show batches with target cost between 70 to 90 in April'): ")
result = filter_batches_by_query(user_query)

# 🖨️ Show result
if not result.empty:
    display(result)
else:
    print("No matching records found.")

Enter your query (e.g., 'Show batches with target cost between 70 to 90 in April'):  Show batches with target cost between 70 to 90 in April


,Batch_ID,Total_Raw_Material_Cost,Output_Units,Output_Date,Target_Cost_Per_Unit,Cost_Per_Unit,Variance_%,Variance_Flag,Month
0,B100,8445.64,1220,2025-04-05,75.19,6.922656,-90.793116,High Variance,April
1,B101,8896.35,978,2025-04-06,79.15,9.096472,-88.507300,High Variance,April
2,B102,19480.44,831,2025-04-07,72.90,23.442166,-67.843394,High Variance,April
5,B105,10201.62,1336,2025-04-10,82.29,7.635943,-90.720691,High Variance,April


### Result for input Query " Show batches with variance between -70 to -50 in April "

In [ ]:
# Interactive input in Jupyter
user_query = input("Enter your query (e.g., 'Show batches with variance between -70 to -50 in April'): ")
result = filter_batches_by_query(user_query)

# 🖨️ Show result
if not result.empty:
    display(result)
else:
    print("No matching records found.")

Enter your query (e.g., 'Show batches with variance between -70 to -50 in April'):  Show batches with variance between -70 to -50 in April


,Batch_ID,Total_Raw_Material_Cost,Output_Units,Output_Date,Target_Cost_Per_Unit,Cost_Per_Unit,Variance_%,Variance_Flag,Month
2,B102,19480.44,831,2025-04-07,72.9,23.442166,-67.843394,High Variance,April


### Result for input Query" Batches with high variance in April "

In [ ]:
# Interactive input in Jupyter
user_query = input("Enter your query (e.g., 'Batches with high variance in April'): ")
result = filter_batches_by_query(user_query)

# 🖨️ Show result
if not result.empty:
    display(result)
else:
    print("No matching records found.")

Enter your query (e.g., 'Batches with high variance in April'):  Batches with high variance in April


,Batch_ID,Total_Raw_Material_Cost,Output_Units,Output_Date,Target_Cost_Per_Unit,Cost_Per_Unit,Variance_%,Variance_Flag,Month
0,B100,8445.64,1220,2025-04-05,75.19,6.922656,-90.793116,High Variance,April
1,B101,8896.35,978,2025-04-06,79.15,9.096472,-88.507300,High Variance,April
2,B102,19480.44,831,2025-04-07,72.90,23.442166,-67.843394,High Variance,April
3,B103,5783.22,1084,2025-04-08,61.32,5.335074,-91.299619,High Variance,April
4,B104,6635.66,1386,2025-04-09,61.58,4.787633,-92.225343,High Variance,April
5,B105,10201.62,1336,2025-04-10,82.29,7.635943,-90.720691,High Variance,April
